In [1]:
import os
import re
import string

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

'''from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
'''
from sklearn.feature_extraction.text import CountVectorizer

### Importing data

In [2]:
import boto3
import pandas as pd
s3 = boto3.client('s3')

with open('balanced_reviews.csv', 'wb') as data:
    s3.download_fileobj('dlfinalproject', 'balanced_reviews.csv', data)
df = pd.read_csv('balanced_reviews.csv')

In [3]:
# Importing reviews
s3 = boto3.resource('s3')
s3.meta.client.upload_file('my project_CNN_GloVe.ipynb', 'dlfinalproject', 'my project.ipynb')

In [4]:
s3 = boto3.client('s3')
with open('glove.6B.300d.txt', 'wb') as data:
    s3.download_fileobj('dlfinalproject', 'glove.6B.300d.txt', data)

In [5]:
#! pip install smart-open

In [6]:
# Uploading Glove
from smart_open import smart_open
def loadGloveModel(glove_s3_path = 's3://dlfinalproject/glove.6B.300d.txt'):
    """ Loads word vectors into a dictionary."""  
    word_vecs = {}
# stream lines from an S3 object
    for line in smart_open(glove_s3_path, 'rb'):
        splitLine = line.decode('utf8').split()
        word = splitLine[0]
        word_vecs[word] = np.array([float(val) for val in splitLine[1:]])
    return word_vecs

In [7]:
word_vecs = loadGloveModel()

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [8]:
print(len(word_vecs.keys()))

400001


In [9]:
df.head()

,Unnamed: 0,business_id,cool,date,funny,review_id,stars,text,useful,user_id,fun_bin
0,17,cHdJXLlKNWixBXpDwEGb_A,1,2015-04-01 16:30:00,7,6BnQwlxRn7ZuWdzninM9sQ,3,I love chinese food and I love mexican food. W...,1,JSrP-dUmLlwZiI7Dp3PQ2A,1
1,21,Mem13A3C202RzT53npn4NA,9,2017-05-13 10:41:43,6,IPw8yWiyqnfBzzWmypUHgg,5,If you are looking for the best pierogies in P...,9,5JVY32_bmTBfIGpCCsnAfw,1
2,62,SU56w479vUfFHsvmvQIf7A,6,2016-07-25 03:55:20,5,E4LqIZ7DJd_R4ZHSNKx4RQ,4,So good! They didn't make it to 5 stars due to...,7,DoRCeCcJbrsM2BiAKj3trA,1
3,126,tjAeaGdxf7I4xN9M7wGJNQ,4,2014-07-13 14:32:56,5,TaoaX7MqCujFRNaJBns2Sw,5,While the prices are a bit high for a make-you...,8,x37OyP--VEFE5p-xreplYA,1
4,246,FhIeCF6QrsLaRvAeu0oEPQ,4,2013-06-24 06:42:29,5,3Qc49B7dA0ONmCxrn5iwCQ,2,OVERALL: The food isn't good (I explain below)...,13,2k8OVAPxlXHsA5X6EIoQpQ,1


In [10]:
df_reviews = df[['funny','text', 'fun_bin']]

In [11]:
pd.set_option('display.max_colwidth', -1)
df_reviews.head()

,funny,text,fun_bin
0,7,"I love chinese food and I love mexican food. What can go wrong? A couple of things. First things first, this place is more of a ""rice bowl"" kind of place. I thought it was going to be more diverse as far as the menu goes, but its mainly rice bowls you get with different kinds of meats. The ordering was a little confusing at first, but one of the employees helped us out and I got the 2-item bowl and got the jade chicken and hengrenade chicken with all rice(jerk). I also ordered a jade chicken quesadilla on the side.\n\nI'm gonna admit, this place looks kinda dirty. I don't think Arizona uses those health department letter grade system like California does, but if I were to just judge by how it looked inside, i'd give it a ""C"" grade lol. We waited for about 15 minutes or so and finally got our food. We took it to go and ate at our hotel room. \n\nMmmm... the food was just alright. The jade chicken was nothing special. It tasted like any generic chinese fast food orange chicken/sesame chicken variant. The hengrenade chicken, although was the less spicier version of the jerk chicken, was still pretty spicy for me. Just be warned the jerk chicken is super spicy. If you aren't sure, ask for a sample at the restaurant before ordering, but it was way too spicy for me. \n\nThe jade chicken quesadilla was decent, but nothing special. Just imagine orange chicken in between a tortilla and cheese. A friend of mine ordered a jade chicken burrito and we were confused when we pulled it out of the bag because it was literally the size of Mcdonald's apple pie. If you order the burrito, be warned that it's a burrito for gnomes and smurfs, but he said it was tasty. \n\nThey provide a snicker doodle sugar cookie for each meal and it was decent, again nothing special. \n\nNot gonna lie, the next day my stomach felt like a little mexican dude and chinese dude were wrestling and throwing molotov cocktails inside. I used the bathroom like 5 times. I don't recommend eating this place if you have a lot to do the next day.",1
1,6,"If you are looking for the best pierogies in Pittsburgh, this is your place. There are a few small tables outside but most of the business is carry out. Pierogies Plus wins Best Pierogies every year. Why? Because the owner is from Poland and she is making the real deal pierogies. The best part is that they are hand pinched by a group of older Polish and Hungarian women. \nThe biggest seller is potato and cheese but they sell many flavors. They are like plump pillows of softness. You can buy them buy the dozen. You can get them cold to take home and freeze or warm and ready to eat. The warm ones are served with butter and onions. It's definitely a comfort food. The best part is that they ship internationally. Yes, they are that good.",1
2,5,"So good! They didn't make it to 5 stars due to the prices are a bit high for the amount of food and the location is a bit unsavory. \nThe decor and atmosphere was surprisingly nice, from the outside I expected to be more run down inside. The staff was very nice. We were surprised how empty the dining room was for a Friday evening.\nWe got Vegetable Samosas to start then ordered Chicken Tikka Masala, Lamb Rogan Josh, rice and plain Naan. Our only complaint was the lamb could've been more tender but everything was flavorful and delicious. \nI would definitely go again if given the chance.",1
3,5,"While the prices are a bit high for a make-your-own pizza, the taste makes up for it. I love going to Seventh Street market, sitting Not Just Coffee and having a drink while waiting for delicious fresh made pizza from Pure. I've taken this to go as well as eaten inside the market, and I can say that the pizza doesn't do well reheated. So try to eat it fresh while there if possible.\n\nIf one of their specialty pizzas sounds good to you, go for it, as those are definitely a better deal for the amount of toppings you get for the money. I wanted what I wanted, though, so I en

### Pre-processing reviews

In [15]:
from string import punctuation
import spacy
import string
import re
import numpy as np
from spacy.symbols import ORTH
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence 
#from spacymoji import Emoji

In [16]:
#Run two commands below and then restart the kernel
#!pip install --upgrade pip

In [17]:
#!pip install -U spacy[cuda92]

In [18]:
#!python3 -m spacy download en_core_web_sm

In [19]:
import spacy

spacy.prefer_gpu()
spacy.load("en_core_web_sm")

In [20]:
import pandas as pd
import numpy as np
#import nltk
#from nltk.corpus import stopwords
#from nltk.stem import SnowballStemmer

from string import punctuation


import numpy as np 
import pandas as pd 
import os
import spacy
import string
import re
import numpy as np
from spacy.symbols import ORTH
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence 

In [21]:

re_br = re.compile(r'<\s*br\s*/?>', re.IGNORECASE)
def sub_br(x): return re_br.sub("\n", x)

#nlp = spacy.load("en")
#spacy.load('en')
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS



def clean_text(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()

    # Clean the text
   # text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=\(\)]", " ", text) # keep punctuatuin, numnbers and letters
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " . ", text) #Add space to the dot
    text = re.sub(r"!", " ! ", text) #Add space to the exclamation sign
    text = re.sub(r":", " :", text) #Add space before : sign
    text = re.sub(r";", " ;", text) #Add space before ; sign
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    #text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    #text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    # find emojis
    emoji_list = []
    '''
    for word in text.split():
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)
    emoji_list'''
    #text = text.split()

    return text

my_tok = spacy.load('en_core_web_sm')
#emoji = Emoji(my_tok)
#my_tok.add_pipe(emoji, first=True)
def spacy_tok(x): return [tok.text for tok in my_tok.tokenizer(clean_text(x))]

def remove_stop_words(tokens): return [tok for tok in tokens if tok not in spacy_stopwords]


### Building a vocabulary

In [121]:
from collections import Counter
counts = Counter()
for sent in df_reviews['text']:
    try:
        counts.update(remove_stop_words(spacy_tok(sent)))
    except:
        pass

In [122]:
#counts

In [123]:
print(len(counts.keys()))

133539


In [114]:
# Vocabulary
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)

In [115]:
# WHat is the 99% quantile of  length of the sentence?

df_reviews['len_text'] = df_reviews['text'].apply(lambda x: len(x.split()))


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [26]:
df_reviews['len_text'].quantile(0.95)

496.0

### Splitting into train and validation sets

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(df_reviews['text'], df_reviews['fun_bin'], test_size=0.2, random_state=42)
    

In [29]:
X_train.reset_index(inplace=True, drop=True)
X_valid.reset_index(inplace=True, drop=True)

In [30]:
y_train.reset_index(inplace=True, drop=True)
y_valid.reset_index(inplace=True, drop=True)

In [31]:
y_train.shape

(120401,)

### Embedding Layer

In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [120]:
counts2=counts.copy()

In [124]:
def delete_rare_words(word_vecs, counts, min_df=0):
    """ Deletes rare words from data_vocab
    
    Deletes words from data_vocab if they are not in word_vecs
    and don't have at least min_df occurrencies in data_vocab.
    """
    counts2 = counts.copy()
    words_delete = []
    for word in counts2:
        if counts2[word] < min_df and word not in word_vecs:
            words_delete.append(word)
    for word in words_delete: counts2.pop(word)
    return counts2

In [125]:
len(counts.keys())

133539

In [130]:
data_vocab = delete_rare_words(word_vecs, counts, min_df=1)


In [131]:
len(data_vocab.keys())

133539

In [140]:
def create_embedding_matrix(word_vecs, data_vocab, min_df=1, D=300):
    """Creates embedding matrix from word vectors. """
    data_vocab = delete_rare_words(word_vecs, data_vocab, min_df)
    V = len(data_vocab.keys()) + 2
    vocab2index = {}
    W = np.zeros((V, D), dtype="float32")
    vocab = ["", "UNK"]
    # adding a vector for padding
    W[0] = np.zeros(D, dtype='float32')
    # adding a vector for rare words 
    W[1] = np.random.uniform(-0.25, 0.25, D)
    vocab2index["UNK"] = 1
    i = 2
    for word in data_vocab:
        if word in word_vecs:
            W[i] = word_vecs[word]
            vocab2index[word] = i
            vocab.append(word)
            i += 1
        else:
            W[i] = np.random.uniform(-0.25,0.25,D)
            vocab2index[word] = i
            vocab.append(word)
            i += 1   
    return W, np.array(vocab), vocab2index

In [141]:
pretrained_weight, vocab, vocab2index = create_embedding_matrix(word_vecs, data_vocab)

In [142]:
len(pretrained_weight) # note that index 0 is for padding

133541

In [143]:
D = 300
V = len(pretrained_weight)
emb = nn.Embedding(V, D)
emb.weight.data.copy_(torch.from_numpy(pretrained_weight))

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1500, -0.2192,  0.1946,  ...,  0.0707, -0.1119, -0.1785],
        [-0.4521, -0.3312, -0.0636,  ...,  0.0389,  0.2431, -0.0287],
        ...,
        [-0.1756,  0.1638,  0.1994,  ...,  0.0169, -0.0949,  0.0164],
        [ 0.0950, -0.1597,  0.1853,  ...,  0.1600, -0.0796, -0.0168],
        [-0.0942, -0.0658, -0.0008,  ...,  0.0448, -0.2433, -0.1585]])

### Encoding training and Validation sets

In [144]:
x_len = np.array([len(x.split()) for x in X_train])

In [145]:
np.percentile(x_len, 95) 

496.0

In [146]:
X_train[0]

'Came by for a drink the other day and it was lackluster. The highlight was an abundance of parking along with an air conditioned seating area...\n\nService by the staff was confusing at best, despite being dressed in all black with their name tags. ...one of the waitresses was named "Drizzle" (...is that a joke?)\n\nThe milk tea with pearls was best described as tea flavored water with pearls that were too soft... There at least was a nice layer of froth on top of my drink...'

In [147]:
# returns the index of the word or the index of "UNK" otherwise
vocab2index.get("drink", vocab2index["UNK"])

226

In [148]:
np.array([vocab2index.get(w, vocab2index["UNK"]) for w in X_train[0].split()])

array([    1,     1,     1,     1,   226,     1,     1,   119,     1,
           1,     1,     1,     1,   379,     1,     1, 14454,     1,
        2756,     1,     1,     1,  3519, 12998,   856,     1,     1,
           1,     1,   196,     1,    29,     1,     1,  1699,     1,
        3938,     1,     1,  1071,     1,     1,     1,     1,     1,
           1,     1,   421,     1,  6154,     1,     1,     1,     1,
           1,     1,  3019,   888,     1, 12949,     1,   134,  4025,
           1,   888,  1451,  1482,     1, 12949,     1,     1,     1,
           1,     1,     1,     1,     1,     1,   193,  4721,     1,
       22211,     1,     1,     1,     1,     1])

In [47]:
#vocab2index

In [149]:

def encode_sentence(s, N=500):
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in s.split()])
    l = min(N, len(enc1))
    enc[:l] = enc1[:l]
    return enc, l

In [150]:
#encode_sentence(X_train[0])

In [51]:
x_train = np.vstack([encode_sentence(x) for x in X_train])
x_train.shape

(120401, 500)

In [52]:
x_val = np.vstack([encode_sentence(x) for x in X_valid])
x_val.shape

(30101, 500)

### Writing a dataset

In [151]:
class YelpDataset(Dataset):
    def __init__(self, df, y, N=500, padding_start=True):
        self.df = df
        self.X = [encode_sentence(sent,  N) for sent in self.df]
        self.y = y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x ,s = self.X[idx]
        return x, s, self.y[idx]


In [152]:
train_ds =  YelpDataset(X_train, y_train, padding_start=False)
valid_ds =  YelpDataset(X_valid, y_valid, padding_start=False)


In [155]:
neg=[]
i=0
for x,s, y in train_ds:
    if s <=0:
        neg.append(i)
    i+=1

neg

[]

In [156]:
#train_ds[0]

In [157]:
batch_size = 500
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size)

In [158]:
### CNN with pretrained embeddings

In [159]:

class SentenceCNN(nn.Module):
    
    def __init__(self, V, D, glove_weights):
        super(SentenceCNN, self).__init__()
        self.glove_weights = glove_weights
        self.embedding = nn.Embedding(V, D, padding_idx=0)
        self.embedding.weight.data.copy_(torch.from_numpy(self.glove_weights))
        self.embedding.weight.requires_grad = False ## freeze embeddings

        self.conv_3 = nn.Conv1d(in_channels=D, out_channels=100, kernel_size=3)
        self.conv_4 = nn.Conv1d(in_channels=D, out_channels=100, kernel_size=4)
        self.conv_5 = nn.Conv1d(in_channels=D, out_channels=100, kernel_size=5)
        
        self.dropout = nn.Dropout(p=0.5)
        self.fc = nn.Linear(300, 1)
        
    def forward(self, x):
        x = self.embedding(x)
        x = x.transpose(1,2)
        x3 = F.relu(self.conv_3(x))
        x4 = F.relu(self.conv_4(x))
        x5 = F.relu(self.conv_5(x))
        x3 = nn.MaxPool1d(kernel_size = 498)(x3)
        x4 = nn.MaxPool1d(kernel_size = 497)(x4)
        x5 = nn.MaxPool1d(kernel_size = 496)(x5)
        out = torch.cat([x3, x4, x5], 2)
        out = out.view(out.size(0), -1)
        out = self.dropout(out)
        return self.fc(out)

In [160]:
V = len(pretrained_weight) #vocabulary size
D = 300 # embedding size
N = 500 # max length size
model = SentenceCNN(V, D, glove_weights=pretrained_weight).cuda()

In [161]:
# testing the model
x = x_train[:10]
print(x.shape)
x = torch.LongTensor(x)

(10, 500)


In [162]:

y_hat = model(x)
y_hat.size()

torch.Size([10, 1])

In [163]:
def save_model(m, p): torch.save(m.state_dict(), p)
    
def load_model(m, p): m.load_state_dict(torch.load(p))


In [164]:
def val_metrics(m, val_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    for x, s, y in val_dl:
        x = x.long().cuda()
        y = y.float().cuda().unsqueeze(1)
        y_hat = model(x)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        y_pred = y_hat > 0
        correct += (y_pred.float() == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
    return sum_loss/total, correct/total

In [165]:
def train_epocs(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    model.train()
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, s, y in train_dl:
            x = x.long().cuda()
            y = y.float().cuda()
            y_pred = model(x)
            optimizer.zero_grad()
            loss = F.binary_cross_entropy_with_logits(y_pred, y.unsqueeze(1))
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc = val_metrics(model, val_dl)
        #if i % 1 == 1:
        print("train loss %.3f val loss %.3f and val accuracy %.3f" % (sum_loss/total, val_loss, val_acc))

In [166]:
batch_size = 500
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(valid_ds, batch_size=batch_size)

In [90]:
#x,s,y = next(iter(train_dl))

In [ ]:
V = len(pretrained_weight) #vocabulary size
D = 300 # embedding size
N = 500 # max length size

In [175]:
model = SentenceCNN(V, D, glove_weights=pretrained_weight).cuda()

In [176]:
train_epocs(model, epochs=5, lr=0.005 )

train loss 0.545 val loss 0.496 and val accuracy 0.764
train loss 0.512 val loss 0.494 and val accuracy 0.765
train loss 0.498 val loss 0.492 and val accuracy 0.771
train loss 0.490 val loss 0.521 and val accuracy 0.760
train loss 0.477 val loss 0.492 and val accuracy 0.767


In [177]:
#train_epocs(model, epochs=10, lr=0.001)

### Unfreezing embeddings

In [178]:
model.embedding.weight.requires_grad = True

In [179]:
parameters = filter(lambda p: p.requires_grad, model.parameters())
print([p.size() for p in parameters])

[torch.Size([133541, 300]), torch.Size([100, 300, 3]), torch.Size([100]), torch.Size([100, 300, 4]), torch.Size([100]), torch.Size([100, 300, 5]), torch.Size([100]), torch.Size([1, 300]), torch.Size([1])]


In [180]:
train_epocs(model, epochs=5, lr=0.001)

train loss 0.441 val loss 0.492 and val accuracy 0.773
train loss 0.394 val loss 0.502 and val accuracy 0.771
train loss 0.351 val loss 0.523 and val accuracy 0.768
train loss 0.307 val loss 0.560 and val accuracy 0.764
train loss 0.272 val loss 0.591 and val accuracy 0.759


In [181]:
#train_epocs(model, epochs=10, lr=0.005)